In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pylab as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, DoubleType, StringType, LongType
from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, from_unixtime
%matplotlib inline

Spark must load the JARs that contain the Pravega Hadoop Connector, Pravega Client, and UTF8StringSerializer.

In [2]:
conf = SparkConf()
conf.set("spark.jars", "/notebooks/work/iiotdemo-sparkprocessor-0.2.0-SNAPSHOT-all.jar")
conf.setMaster('local[2]')

In [3]:
sc = SparkContext.getOrCreate(conf=conf)
sql_sc = SQLContext(sc)

Create an RDD of strings from JSON events in a Pravega stream.

In [15]:
rdd = sc.newAPIHadoopRDD(
    inputFormatClass='io.pravega.connectors.hadoop.PravegaInputFormat',
    keyClass='io.pravega.connectors.hadoop.EventKey',
    valueClass='java.lang.String',
    conf={
        'pravega.scope': 'iot1',
        'pravega.stream': 'data',
        'pravega.uri': 'tcp://10.246.21.230:9090',
        'pravega.deserializer': 'io.pravega.example.iiotdemo.sparkprocessor.UTF8StringSerializer',
    },
)

Read Pravega events without specifying a schema. This performs two passes on the data.

In [5]:
%%time
df = sql_sc.read.json(rdd.values())
df.show()

+---------+----------+-----------+------------------+-------------+------------------+-----------------+
|device_id|event_type|remote_addr|      temp_celsius|    timestamp|        vibration1|       vibration2|
+---------+----------+-----------+------------------+-------------+------------------+-----------------+
|     0002|      temp|  127.0.0.1|23.447554062011953|1526594516113|              null|             null|
|     0001|      temp|  127.0.0.1| 37.22421743210678|1526594516112|              null|             null|
|     0002| vibration|  127.0.0.1|              null|1526594516113|2965.9755406201193|6031.951081240239|
|     0001| vibration|  127.0.0.1|              null|1526594516112| 3103.742174321068|6307.484348642136|
|     0002|      temp|  127.0.0.1| 23.39155667180122|1526594517127|              null|             null|
|     0002| vibration|  127.0.0.1|              null|1526594517127|2965.4155667180116|6030.831133436023|
|     0001|      temp|  127.0.0.1| 37.14044473638414|15

View the generated schema.

In [6]:
df.schema

StructType(List(StructField(device_id,StringType,true),StructField(event_type,StringType,true),StructField(remote_addr,StringType,true),StructField(temp_celsius,DoubleType,true),StructField(timestamp,LongType,true),StructField(vibration1,DoubleType,true),StructField(vibration2,DoubleType,true)))

Read Pravega events with a schema. We also cache the result in memory.

In [16]:
%%time
schema = StructType([
        StructField('timestamp', LongType(), True),
        StructField('event_type', StringType(), True),
        StructField('device_id', StringType(), True),
        StructField('temp_celsius', DoubleType(), True),
        StructField('vibration1', DoubleType(), True),
        StructField('vibration2', DoubleType(), True),
    ])
df = sql_sc.read.schema(schema).json(rdd.values()).cache()
df.show()

+-------------+----------+---------+------------------+------------------+-----------------+
|    timestamp|event_type|device_id|      temp_celsius|        vibration1|       vibration2|
+-------------+----------+---------+------------------+------------------+-----------------+
|1526594516113|      temp|     0002|23.447554062011953|              null|             null|
|1526594516112|      temp|     0001| 37.22421743210678|              null|             null|
|1526594516113| vibration|     0002|              null|2965.9755406201193|6031.951081240239|
|1526594516112| vibration|     0001|              null| 3103.742174321068|6307.484348642136|
|1526594517127|      temp|     0002| 23.39155667180122|              null|             null|
|1526594517127| vibration|     0002|              null|2965.4155667180116|6030.831133436023|
|1526594517437|      temp|     0001| 37.14044473638414|              null|             null|
|1526594517437| vibration|     0001|              null|3102.9044473638

In [8]:
df.filter('event_type="temp"').show()

+-------------+----------+---------+------------------+----------+----------+
|    timestamp|event_type|device_id|      temp_celsius|vibration1|vibration2|
+-------------+----------+---------+------------------+----------+----------+
|1526594516113|      temp|     0002|23.447554062011953|      null|      null|
|1526594516112|      temp|     0001| 37.22421743210678|      null|      null|
|1526594517127|      temp|     0002| 23.39155667180122|      null|      null|
|1526594517437|      temp|     0001| 37.14044473638414|      null|      null|
|1526594518141|      temp|     0002|23.339366447471768|      null|      null|
|1526594518762|      temp|     0001| 37.07423386711112|      null|      null|
|1526594519155|      temp|     0002|23.290990950746018|      null|      null|
|1526594520087|      temp|     0001| 37.02570011068423|      null|      null|
|1526594520169|      temp|     0002|23.246437283613528|      null|      null|
+-------------+----------+---------+------------------+---------

In [17]:
df.count()

272

In [18]:
df = df.withColumn('timestamp_str', from_unixtime(df['timestamp'] / 1000.0))
df.show()

+-------------+----------+---------+------------------+------------------+-----------------+-------------------+
|    timestamp|event_type|device_id|      temp_celsius|        vibration1|       vibration2|      timestamp_str|
+-------------+----------+---------+------------------+------------------+-----------------+-------------------+
|1526594516113|      temp|     0002|23.447554062011953|              null|             null|2018-05-17 22:01:56|
|1526594516112|      temp|     0001| 37.22421743210678|              null|             null|2018-05-17 22:01:56|
|1526594516113| vibration|     0002|              null|2965.9755406201193|6031.951081240239|2018-05-17 22:01:56|
|1526594516112| vibration|     0001|              null| 3103.742174321068|6307.484348642136|2018-05-17 22:01:56|
|1526594517127|      temp|     0002| 23.39155667180122|              null|             null|2018-05-17 22:01:57|
|1526594517127| vibration|     0002|              null|2965.4155667180116|6030.831133436023|2018

In [19]:
df.sort('timestamp').show()

+-------------+----------+---------+------------------+------------------+-----------------+-------------------+
|    timestamp|event_type|device_id|      temp_celsius|        vibration1|       vibration2|      timestamp_str|
+-------------+----------+---------+------------------+------------------+-----------------+-------------------+
|1526594516112|      temp|     0001| 37.22421743210678|              null|             null|2018-05-17 22:01:56|
|1526594516112| vibration|     0001|              null| 3103.742174321068|6307.484348642136|2018-05-17 22:01:56|
|1526594516113|      temp|     0002|23.447554062011953|              null|             null|2018-05-17 22:01:56|
|1526594516113| vibration|     0002|              null|2965.9755406201193|6031.951081240239|2018-05-17 22:01:56|
|1526594517127|      temp|     0002| 23.39155667180122|              null|             null|2018-05-17 22:01:57|
|1526594517127| vibration|     0002|              null|2965.4155667180116|6030.831133436023|2018

In [20]:
sql_sc.registerDataFrameAsTable(df, 'rawData')

Run SQL queries.

In [21]:
sql_sc.sql('select * from rawData where event_type="vibration" order by vibration2 desc').show()

+-------------+----------+---------+------------+------------------+-----------------+-------------------+
|    timestamp|event_type|device_id|temp_celsius|        vibration1|       vibration2|      timestamp_str|
+-------------+----------+---------+------------+------------------+-----------------+-------------------+
|1526597031021| vibration|     0005|        null|3608.0137830175254|7316.027566035051|2018-05-17 22:43:51|
|1526597031808| vibration|     0005|        null|3604.7559106554054|7309.511821310811|2018-05-17 22:43:51|
|1526597032595| vibration|     0005|        null| 3601.368713491468|7302.737426982936|2018-05-17 22:43:52|
|1526597033382| vibration|     0005|        null|3597.8536024867435|7295.707204973487|2018-05-17 22:43:53|
|1526597034169| vibration|     0005|        null| 3594.212032820148|7288.424065640296|2018-05-17 22:43:54|
|1526597034956| vibration|     0005|        null| 3590.445521268653|7280.891042537306|2018-05-17 22:43:54|
|1526597035743| vibration|     0005| 

In [22]:
%%time
sql_sc.sql("""
select device_id, event_type, count(*) as num_events, from_unixtime(max(timestamp)/1000) as last_timestamp
from rawData
group by device_id, event_type
""").show()

+---------+----------+----------+-------------------+
|device_id|event_type|num_events|     last_timestamp|
+---------+----------+----------+-------------------+
|     0003| vibration|        10|2018-05-17 22:44:04|
|     0002| vibration|        20|2018-05-17 22:44:03|
|     0006|      temp|        11|2018-05-17 22:44:04|
|     0007|      temp|        13|2018-05-17 22:44:04|
|     0010|      temp|        10|2018-05-17 22:44:04|
|     0008| vibration|        10|2018-05-17 22:44:03|
|     0001| vibration|        20|2018-05-17 22:44:04|
|     0004|      temp|        10|2018-05-17 22:44:03|
|     0003|      temp|        10|2018-05-17 22:44:04|
|     0002|      temp|        20|2018-05-17 22:44:03|
|     0004| vibration|        10|2018-05-17 22:44:03|
|     0005|      temp|        18|2018-05-17 22:44:04|
|     0007| vibration|        13|2018-05-17 22:44:04|
|     0009| vibration|        14|2018-05-17 22:44:04|
|     0010| vibration|        10|2018-05-17 22:44:04|
|     0009|      temp|      